In [0]:
%run ./00_config

In [0]:
# --- CELDA 2 ---
from pyspark.sql.functions import col, to_timestamp, current_timestamp

try:
    print("⏳ Iniciando procesamiento de la Capa Silver...")
    
    # 1. LECTURA DE CAPA BRONZE
    # Databricks leerá la variable BRONCE_TAXI_TABLE de tu archivo 00_config
    df_viajes = spark.table(BRONCE_TAXI_TABLE)
    
    # 2. LIMPIEZA Y NORMALIZACIÓN (Reglas de Calidad Senior)
    print("🧹 Limpiando y estandarizando datos (Casteos y Filtros)...")
    df_silver = df_viajes.withColumn("pickup_datetime", to_timestamp(col("pickup_datetime"))) \
                         .withColumn("dropoff_datetime", to_timestamp(col("dropoff_datetime"))) \
                         .withColumn("base_passenger_fare", col("base_passenger_fare").cast("decimal(10,2)")) \
                         .withColumn("tolls", col("tolls").cast("decimal(10,2)")) \
                         .filter(col("PULocationID").isNotNull()) \
                         .filter(col("dropoff_datetime") > col("pickup_datetime")) # Filtramos viajes ilógicos

    # 3. ENRIQUECIMIENTO (JOIN con Zonas)
    print("🗺️ Enriqueciendo datos con el catálogo de zonas...")
    
    # Validación: Verificamos si la tabla de zonas existe antes de cruzar
    if spark.catalog.tableExists(BRONCE_ZONES_TABLE):
        df_zonas = spark.table(BRONCE_ZONES_TABLE)
        
        # Hacemos un LEFT JOIN para no perder viajes si la zona no aparece en el catálogo
        df_final_silver = df_silver.join(
            df_zonas, 
            df_silver.PULocationID == df_zonas.LocationID, 
            "left"
        ).select(
            df_silver["*"],
            col("Borough").alias("pickup_borough"),
            col("Zone").alias("pickup_zone")
        )
    else:
        print(f"⚠️ Tabla '{BRONCE_ZONES_TABLE}' no encontrada. Omitiendo JOIN de zonas por ahora.")
        df_final_silver = df_silver

    # 4. AUDITORÍA Y PERSISTENCIA
    # Agregamos la marca de tiempo exacta de cuándo pasó por la capa Silver
    df_final_silver = df_final_silver.withColumn("silver_processing_timestamp", current_timestamp())

    print(f"💾 Guardando datos refinados en la tabla: {SILVER_TAXI_TABLE}...")
    df_final_silver.write.mode("overwrite") \
                   .option("overwriteSchema", "true") \
                   .saveAsTable(SILVER_TAXI_TABLE)

    print("✅ ¡VICTORIA! Capa Silver completada con éxito.")
    
    # Mostramos el esquema resultante y los primeros 5 registros
    df_final_silver.printSchema()
    display(df_final_silver.limit(5))

except Exception as e:
    print(f"❌ Error en Capa Silver: {str(e)}")

In [0]:
%sql
select count(1) from silver_nyc_taxi